### Libraries

In [274]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
from psutil import virtual_memory    
import datetime                          

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

### GPU

In [214]:
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
ram_gb = virtual_memory().total / 1e9
print(f'{gpu.s} with {round(ram_gb,1)} GB available RAM.')
!nvcc --version

'nvidia-smi' is not recognized as an internal or external command, operable program or batch file. with 8.5 GB available RAM.


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


### Load Data

In [215]:
data_dir = os.path.abspath('../data')

# Non-augmented dataset
df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k.csv'), low_memory=False)
AUGMENTATION = 'None'

# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE'

# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE-NC'

# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# AUGMENTATION = 'RealTabFormer'

# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
# AUGMENTATION = 'GReaT'

# Test data for all datasets
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

In [216]:
# list columns of df_train
df_train.columns

Index(['arp.opcode', 'arp.hw.size', 'icmp.checksum', 'icmp.seq_le',
       'icmp.unused', 'http.content_length', 'http.request.method',
       'http.referer', 'http.request.version', 'http.response',
       'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum',
       'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn',
       'tcp.connection.synack', 'tcp.flags', 'tcp.flags.ack', 'tcp.len',
       'tcp.seq', 'udp.stream', 'udp.time_delta', 'dns.qry.name',
       'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission',
       'dns.retransmit_request', 'dns.retransmit_request_in',
       'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags',
       'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msgtype',
       'mqtt.proto_len', 'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len',
       'mqtt.ver', 'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id',
       'Attack_label', 'Attack_type'],
      dtype='object')

### Data Preparation

In [217]:
# Drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [218]:
# Extract categorical features
categorical_features = X_train.select_dtypes(include="object").columns

# Get the unique values of all categorical columns
for col in X_train[categorical_features].columns:
        unique_values = X_train[col].unique()
        print(f'{col}: \n{unique_values}\n')

http.request.method: 
['0.0' '0' 'GET' 'POST' 'TRACE' 'OPTIONS' 'SEARCH' 'PROPFIND' 'PUT']

http.referer: 
['0.0' '0' '127.0.0.1'
 '() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }'
 'TESTING_PURPOSES_ONLY']

http.request.version: 
['0.0' '0' 'HTTP/1.1' 'HTTP/1.0' 'script>alert(1)/script><\\" HTTP/1.1'
 '/etc/passwd|?data=Download HTTP/1.1'
 '-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1'
 '-al&_PHPLIB[libdir]=http://cirt.net/rfiinc.txt?? HTTP/1.1' '-a HTTP/1.1'
 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1'
 "name=a><input name=i value=XSS>&lt;script>alert('Vulnerable')</script> HTTP/1.1"
 'By Dr HTTP/1.1' '> HTTP/1.1']

dns.qry.name.len: 
['0.0' '1.0' '0' '2.debian.pool.ntp.org' '1.debian.pool.ntp.org'
 '3.debian.pool.ntp.org' '0.debian.pool.ntp.org' 'raspberrypi.local']

mqtt.conack.flags: 
['0.0' '0' '0x00000000' '1574358' '1461589' '1461383' '1574359']

mqtt.protoname: 
['0.0' '0' 'MQTT']

mqtt.topic: 
['0.0' '0' 'Temperature_and

In [219]:
# Concatenate X_train and X_test
X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# Apply one-hot encoding (get_dummies)
X_comb_enc = pd.get_dummies(X_comb, dtype='int8')

# Split back into X_train and X_test
X_train_enc, X_test_enc = train_test_split(
    X_comb_enc, test_size=len(X_test), random_state=42)

# Print the shape of X_train_enc and X_test_enc
print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (536515, 53), X_test_enc shape: (381934, 53)


In [220]:
# Drop columns categorical_features from X_train and X_test 
X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)

# Concatenate X_train and X_train_enc
X_train = pd.concat([X_train.reset_index(), X_train_enc.reset_index()], axis=1)
X_test = pd.concat([X_test.reset_index(), X_test_enc.reset_index()], axis=1)

# Print the shape of X_train and X_test
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (536515, 92), X_test shape: (381934, 92)


#### Label Encoding

In [221]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


#### One-hot encoding of labels

In [222]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

#### Standardization of Data

In [170]:
# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)

# Mean and standard deviation of X_train and X_test
print(f'X_train mean: {X_train.mean():.2f}, X_train std: {X_train.std():.2f}')
print(f'X_test  mean:  {X_test.mean():.2f}, X_test  std: {X_test.std():.2f}')

X_train mean: -0.00, X_train std: 0.94
X_test  mean:  0.03, X_test  std: 1.08


### Model Training

In [174]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint('best_model_multiclass.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [175]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Train the model
history = model.fit(X_train, 
                    y_train, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

Epoch 1/100
1048/1048 [==============================] - 16s 11ms/step - loss: 0.9142 - accuracy: 0.6781 - lr: 0.0010
Epoch 2/100
   1/1048 [..............................] - ETA: 14s - loss: 0.7117 - accuracy: 0.7441

c:\ambientes_virtuais_py\data_augment\data_augment\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1048/1048 [==============================] - 13s 13ms/step - loss: 0.6994 - accuracy: 0.7374 - lr: 0.0010
Epoch 3/100
1048/1048 [==============================] - 14s 13ms/step - loss: 0.6382 - accuracy: 0.7584 - lr: 0.0010
Epoch 4/100
1048/1048 [==============================] - 15s 14ms/step - loss: 0.6158 - accuracy: 0.7654 - lr: 0.0010
Epoch 5/100
1048/1048 [==============================] - 15s 14ms/step - loss: 0.5999 - accuracy: 0.7704 - lr: 0.0010
Epoch 6/100
1048/1048 [==============================] - 14s 14ms/step - loss: 0.5883 - accuracy: 0.7737 - lr: 0.0010
Epoch 7/100
1048/1048 [==============================] - 13s 13ms/step - loss: 0.5780 - accuracy: 0.7764 - lr: 0.0010
Epoch 8/100
1048/1048 [==============================] - 15s 15ms/step - loss: 0.5702 - accuracy: 0.7788 - lr: 0.0010
Epoch 9/100
1048/1048 [==============================] - 15s 15ms/step - loss: 0.5652 - accuracy: 0.7806 - lr: 0.0010
Epoch 10/100
1048/1048 [==============================] - 15s 15ms/s

### Model Evaluation

In [186]:
# predict probabilities for test set and get the index of the highest probability
predictions = tf.argmax(model.predict(X_test), axis=1)

11936/11936 [==============================] - 23s 2ms/step


In [275]:
# Calculate metrics 
accuracy = metrics.accuracy_score(tf.argmax(y_test, axis=1), predictions)
precision = metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='weighted', zero_division=1)
recall = metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='weighted')
f1_score = metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='weighted')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (Weighted): {:.2f}".format(precision))
print("Recall (Weighted): {:.2f}".format(recall))
print("F1(Weighted): {:.2f}".format(f1_score))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")


Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.44
Precision (Weighted): 0.92
Recall (Weighted): 0.44
F1(Weighted): 0.52
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [265]:
# create dictionary for results
results = {
    "model": "TabNet",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": metrics.accuracy_score(tf.argmax(y_test, axis=1), predictions),
    "precision": metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='weighted'),
    "recall": metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='weighted'),
    "f1": metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='weighted')
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/Tabnet.csv')

#### Confusion Matrix

In [273]:
conf_mat = metrics.confusion_matrix(tf.argmax(y_test, axis=1), predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4353,180,0,81,1,0,0,0,18,0,0,52,87,10,0
DDoS_HTTP,194,5817,0,0,1,0,0,0,3,1,110,3493,6,3,0
DDoS_ICMP,0,0,13483,0,6,12,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,1,0,10007,0,0,0,0,0,0,0,0,1,0,0
DDoS_UDP,0,0,0,0,24600,0,0,1,0,0,0,0,0,0,0
Fingerprinting,0,25,61,23,5,32,0,0,0,0,0,0,0,0,0
MITM,0,0,0,0,1,0,75,0,0,0,0,0,0,0,0
Normal,15898,27967,7,20,2,0,1,86143,98,0,4986,137615,23,16,0
Password,273,897,0,0,0,0,0,2,1553,0,190,6984,0,209,0
